<a href="https://colab.research.google.com/github/cinthiamonteiro/olist-analysis/blob/main/olist_brazilian_ecommerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação dos dados

In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 14.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
from google.colab import files
import os
import pandas as pd
import numpy as np
from unidecode import unidecode


# Importando os dados a partir do google drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/olist_brazilian_ecommerce/data/'

orders = pd.read_csv(path + "olist_orders_dataset.csv")
customers = pd.read_csv(path + "olist_customers_dataset.csv", dtype={'customer_zip_code_prefix': str})
order_items = pd.read_csv(path + "olist_order_items_dataset.csv")
products = pd.read_csv(path + "olist_products_dataset.csv")
sellers = pd.read_csv(path + "olist_sellers_dataset.csv", dtype={'seller_zip_code_prefix': str})
reviews = pd.read_csv(path + "olist_order_reviews_dataset.csv")
payments = pd.read_csv(path + "olist_order_payments_dataset.csv")
location = pd.read_csv(path + "olist_geolocation_dataset.csv", dtype={'geolocation_zip_code_prefix': str})

# Lista dos DataFrames do projeto
tabelas = ['orders','customers','order_items','products','sellers','reviews','payments','location']

# Dicionário com apenas os DataFrames para auxiliar estruturas iterativas
dfs = {name: globals()[name] for name in tabelas if name in globals()}

Mounted at /content/drive


# Análise Exploratória Inicial

## Análise geral

In [ ]:
for name,df in dfs.items():
  print(f'Dataframe: {name}:')
  print(df.info())
  print('-'*100,'\n')


Dataframe: orders:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
None
---------------------------------------------------------------------------------------------------- 

Dataframe: customers:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   

## Checagem de dados duplicados

In [ ]:
for name, df in dfs.items():
  print(f'Dataframe: {name}\n')
  for column in df.columns:
    print(f'{column}: {df[column].duplicated().sum()}')
  print('-'*50,'\n')

Dataframe: orders

order_id: 0
customer_id: 0
order_status: 99433
order_purchase_timestamp: 566
order_approved_at: 8707
order_delivered_carrier_date: 18422
order_delivered_customer_date: 3776
order_estimated_delivery_date: 98982
-------------------------------------------------- 

Dataframe: customers

customer_id: 0
customer_unique_id: 3345
customer_zip_code_prefix: 84447
customer_city: 95322
customer_state: 99414
-------------------------------------------------- 

Dataframe: order_items

order_id: 13984
order_item_id: 112629
product_id: 79699
seller_id: 109555
shipping_limit_date: 19332
price: 106682
freight_value: 105651
-------------------------------------------------- 

Dataframe: products

product_id: 0
product_category_name: 32877
product_name_lenght: 32884
product_description_lenght: 29990
product_photos_qty: 32931
product_weight_g: 30746
product_length_cm: 32851
product_height_cm: 32848
product_width_cm: 32855
-------------------------------------------------- 

Dataframe: s

##Checagem de restrições

In [ ]:
# Premissa: zip_code contem os 5 primeiros digitos do CEP, desse modo, comprimento da variável é igual a 5.
def check_zip_length(df, column_name, expected_length=5):

    invalid_len = df[df[column_name].astype(str).str.len() != expected_length]
    invalid_non_numeric = df[~df[column_name].astype(str).str.isnumeric()]
    print(f"Número de registros inválidos em '{column_name}': {len(invalid_len) + len(invalid_non_numeric)}")

# Checagem para a tabela customers
check_zip_length(customers, 'customer_zip_code_prefix')

# Checagem para a tabela sellers
check_zip_length(sellers, 'seller_zip_code_prefix')

# Checagem para a tabela location
check_zip_length(location, 'geolocation_zip_code_prefix')


Número de registros inválidos em 'customer_zip_code_prefix': 0
Número de registros inválidos em 'seller_zip_code_prefix': 0
Número de registros inválidos em 'geolocation_zip_code_prefix': 0


##Validação de categorias

In [ ]:
import re

# Validação se há typos a serem corrigidos na categoria payment_type

payment_types_df = (
    payments['payment_type']
    .value_counts()
    .reset_index()
)

print("\nCategorias em 'payment_type':\n")
display(payment_types_df)

# Validação se há typos a serem corrigidos na categoria order_status
order_status_df = (
    orders['order_status']
    .value_counts()
    .reset_index()
)

print("\nCategorias em 'order_status':\n")
display(order_status_df)

# Verificação de typos ou discrepâncias no campo de product_category

product_cat = products[['product_category_name']].groupby('product_category_name').count().reset_index()

print("\nCategorias fora do padrão esperado:\n")
display(product_cat[product_cat['product_category_name'].str.contains('[^a-zA-Zá-úÁ-Ú_]', na=False)])


Categorias em 'payment_type':



,payment_type,count
0,credit_card,76795
1,boleto,19784
2,voucher,5775
3,debit_card,1529
4,not_defined,3



Categorias em 'order_status':



,order_status,count
0,delivered,96478
1,shipped,1107
2,canceled,625
3,unavailable,609
4,invoiced,314
5,processing,301
6,created,5
7,approved,2



Categorias fora do padrão esperado:



,product_category_name
15,casa_conforto_2
29,eletrodomesticos_2


#Tratamento de dados

## Tratamento de nulos e duplicados

In [ ]:
# Dataframe: location
# No caso da tabela location, não estamos interessados no endereço exato (e essa informação não seria precisa o suficiente, mesmo que precisássemos dela). Em vez disso, usamos os cinco primeiros dígitos do CEP como chave primária para essa tabela.
location.drop_duplicates(subset=['geolocation_zip_code_prefix'],inplace=True)

# Dataframe: reviews
# Para a tabela reviews, vamos remover as duplicatas, manteremos o ultimo registro (considerando criterio de data/horário) entendendo que o cliente pode ter alterado a nota ou atualizado a avaliação posterior ao envio inicial
reviews.sort_values(by=['review_id', 'review_answer_timestamp'], inplace=True)
reviews.drop_duplicates(subset=['review_id'], keep='last', inplace=True)

# Dataframe: products
# Preencheremos os nulos de categoria indicando a ausência de informação, e para parte dos campos indicaremos essa ausência substitiuindo valores nulos pelo valor 0
products['product_category_name'] = products['product_category_name'].fillna('sem_categoria')
products[['product_description_lenght', 'product_photos_qty', 'product_name_lenght']] = (
    products[['product_description_lenght', 'product_photos_qty', 'product_name_lenght']].fillna(0)
)


##Conversão dos tipos de dado

In [ ]:
# Dataframe: orders
orders[['order_id','customer_id']] = orders[['order_id','customer_id']].astype('string')
orders['order_status'] = orders['order_status'].astype('category')
columns_timestamp = ['order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date']
orders[columns_timestamp] = orders[columns_timestamp].apply(pd.to_datetime,format='%Y-%m-%d %H:%M:%S',errors='coerce')

# Dataframe: customers
customers[['customer_id','customer_unique_id','customer_state','customer_zip_code_prefix']] = customers[['customer_id','customer_unique_id','customer_state','customer_zip_code_prefix']].astype('string')

# Dataframe: order_items
order_items[['order_id','product_id','seller_id']] = order_items[['order_id','product_id','seller_id']].astype('string')
order_items['shipping_limit_date'] = pd.to_datetime(order_items['shipping_limit_date'],format='%Y-%m-%d %H:%M:%S',errors='coerce')

# Dataframe: products
products['product_id'] = products['product_id'].astype('string')
products[['product_description_lenght', 'product_photos_qty', 'product_name_lenght']] = products[['product_description_lenght', 'product_photos_qty', 'product_name_lenght']].astype('int')

# Dataframe: sellers
sellers[['seller_id','seller_state','seller_zip_code_prefix']] = sellers[['seller_id','seller_state','seller_zip_code_prefix']].astype('string')

# Dataframe: reviews
reviews[['review_id','order_id']] = reviews[['review_id','order_id']].astype('string')
columns_timestamp = ['review_answer_timestamp','review_creation_date']
reviews[columns_timestamp] = reviews[columns_timestamp].apply(pd.to_datetime,format='%Y-%m-%d %H:%M:%S',errors='coerce')

# Dataframe: payments
payments['order_id'] = payments['order_id'].astype('string')
payments['payment_type'] = payments['payment_type'].astype('category')

# Dataframe: location
location[['geolocation_city','geolocation_state']] = location[['geolocation_city','geolocation_state']].astype('string')

##Padronização dos dados

In [ ]:
# O output das tabelas location, customers e sellers mostrou a falta de padronização no campo que informa a cidade, nesse sentido iremos padronizar a capitalização e restringir o uso de carcateres especiais

location['geolocation_city'] = location['geolocation_city'].apply(lambda x: unidecode(x.strip().lower())).astype('string')
customers['customer_city'] = customers['customer_city'].apply(lambda x: unidecode(x.strip().lower())).astype('string')
sellers['seller_city'] = sellers['seller_city'].apply(lambda x: unidecode(x.strip().lower())).astype('string')


# Incidencia de categorias com indices (ex.: casa_conforto, casa_conforto_2). E padronizar capitalização de letras e uso de caracteres especiais
products['product_category_name'] = products['product_category_name'].str.replace(r'_\d+$', '', regex=True)
products['product_category_name'] = products['product_category_name'].apply(lambda x: unidecode(x.strip().lower())).astype('string')


##Verificação do tratamento aplicado

In [ ]:
#Vamos verificar primeiramente se o tipo indicado foi corretamente atrelado e se o tratamento de duplicadas/nulos foi efetivo nos casos em que foi necessário
for name, df in dfs.items():
  print(f'Dataframe: {name}\n')
  for column in df.columns:
    print(f'{column}: {df[column].duplicated().sum()} duplicados | {df[column].isnull().sum()} nulos |tipo: {df[column].dtype}')
  print('-'*50,'\n')

Dataframe: orders

order_id: 0 duplicados | 0 nulos |tipo: string
customer_id: 0 duplicados | 0 nulos |tipo: string
order_status: 99433 duplicados | 0 nulos |tipo: category
order_purchase_timestamp: 566 duplicados | 0 nulos |tipo: datetime64[ns]
order_approved_at: 8707 duplicados | 160 nulos |tipo: datetime64[ns]
order_delivered_carrier_date: 18422 duplicados | 1783 nulos |tipo: datetime64[ns]
order_delivered_customer_date: 3776 duplicados | 2965 nulos |tipo: datetime64[ns]
order_estimated_delivery_date: 98982 duplicados | 0 nulos |tipo: datetime64[ns]
-------------------------------------------------- 

Dataframe: customers

customer_id: 0 duplicados | 0 nulos |tipo: string
customer_unique_id: 3345 duplicados | 0 nulos |tipo: string
customer_zip_code_prefix: 84447 duplicados | 0 nulos |tipo: string
customer_city: 95322 duplicados | 0 nulos |tipo: string
customer_state: 99414 duplicados | 0 nulos |tipo: string
-------------------------------------------------- 

Dataframe: order_items


#Criação de tabelas (SQLite)

In [ ]:
import sqlite3

conn = sqlite3.connect('olist.db')
cursor = conn.cursor()

cursor.execute("PRAGMA foreign_keys = ON;")

# Staging

customers.to_sql('stg_customers', conn, index=False, if_exists='replace')
sellers.to_sql('stg_sellers', conn, index=False, if_exists='replace')
products.to_sql('stg_products', conn, index=False, if_exists='replace')
location.to_sql('stg_location', conn, index=False, if_exists='replace')
orders.to_sql('stg_orders', conn, index=False, if_exists='replace')
order_items.to_sql('stg_order_items', conn, index=False, if_exists='replace')
reviews.to_sql('stg_reviews', conn, index=False, if_exists='replace')
payments.to_sql('stg_payments', conn, index=False, if_exists='replace')

conn.commit()

cursor.executescript("""
-- dim_customers
CREATE TABLE IF NOT EXISTS dim_customers (
    customer_unique_id TEXT PRIMARY KEY,
    customer_zip_code_prefix TEXT,
    customer_city TEXT,
    customer_state TEXT,
    geolocation_lat REAL,
    geolocation_lng REAL
);

INSERT OR REPLACE INTO dim_customers
SELECT DISTINCT
    c.customer_unique_id,
    c.customer_zip_code_prefix,
    c.customer_city,
    c.customer_state,
    l.geolocation_lat,
    l.geolocation_lng
FROM stg_customers c
LEFT JOIN stg_location l ON c.customer_zip_code_prefix = l.geolocation_zip_code_prefix;

-- dim_sellers

CREATE TABLE IF NOT EXISTS dim_sellers (
    seller_id TEXT PRIMARY KEY,
    seller_zip_code_prefix TEXT,
    seller_city TEXT,
    seller_state TEXT,
    geolocation_lat REAL,
    geolocation_lng REAL
);

INSERT OR REPLACE INTO dim_sellers
SELECT DISTINCT
    s.seller_id,
    s.seller_zip_code_prefix,
    s.seller_city,
    s.seller_state,
    l.geolocation_lat,
    l.geolocation_lng
FROM stg_sellers s
LEFT JOIN stg_location l ON s.seller_zip_code_prefix = l.geolocation_zip_code_prefix;
;

-- dim_products

CREATE TABLE IF NOT EXISTS dim_products (
    product_id TEXT PRIMARY KEY,
    product_category_name TEXT,
    product_name_lenght INT,
    product_description_lenght INT,
    product_photos_qty INT
);

INSERT OR REPLACE INTO dim_products
SELECT DISTINCT
    product_id,
    product_category_name,
    product_name_lenght,
    product_description_lenght,
    product_photos_qty
FROM stg_products;

-- fact_orders

CREATE TABLE IF NOT EXISTS fact_orders (
    order_id TEXT PRIMARY KEY,
    customer_unique_id TEXT,
    order_status TEXT,
    order_purchase_timestamp DATETIME,
    order_approved_at DATETIME,
    order_delivered_carrier_date DATETIME,
    order_delivered_customer_date DATETIME,
    order_estimated_delivery_date DATETIME,
    FOREIGN KEY (customer_unique_id) REFERENCES dim_customers(customer_unique_id)
);

INSERT OR REPLACE INTO fact_orders
SELECT
    o.order_id,
    c.customer_unique_id,
    o.order_status,
    o.order_purchase_timestamp,
    o.order_approved_at,
    o.order_delivered_carrier_date,
    o.order_delivered_customer_date,
    o.order_estimated_delivery_date
FROM stg_orders o
JOIN stg_customers c ON o.customer_id = c.customer_id;

-- fact_order_items

CREATE TABLE IF NOT EXISTS fact_order_items (
    order_id TEXT,
    order_item_id INTEGER,
    product_id TEXT,
    seller_id TEXT,
    shipping_limit_date DATETIME,
    price REAL,
    freight_value REAL,
    PRIMARY KEY (order_id, order_item_id),
    FOREIGN KEY (order_id) REFERENCES fact_orders(order_id),
    FOREIGN KEY (product_id) REFERENCES dim_products(product_id),
    FOREIGN KEY (seller_id) REFERENCES dim_sellers(seller_id)
);

INSERT OR REPLACE INTO fact_order_items
SELECT
    order_id,
    order_item_id,
    product_id,
    seller_id,
    shipping_limit_date,
    price,
    freight_value
FROM stg_order_items;

-- fact_reviews

CREATE TABLE IF NOT EXISTS fact_reviews (
    review_id TEXT PRIMARY KEY,
    order_id TEXT,
    review_score INTEGER,
    review_comment_title TEXT,
    review_comment_message TEXT,
    review_creation_date DATETIME,
    review_answer_timestamp DATETIME,
    FOREIGN KEY (order_id) REFERENCES fact_orders(order_id)
);

INSERT OR REPLACE INTO fact_reviews
SELECT
    review_id,
    order_id,
    review_score,
    review_comment_title,
    review_comment_message,
    review_creation_date,
    review_answer_timestamp
FROM stg_reviews;

-- fact_payments

CREATE TABLE IF NOT EXISTS fact_payments (
    order_id TEXT,
    payment_sequential INTEGER,
    payment_type TEXT,
    payment_installments INTEGER,
    payment_value REAL,
    PRIMARY KEY (order_id, payment_sequential),
    FOREIGN KEY (order_id) REFERENCES fact_orders(order_id)
);

INSERT OR REPLACE INTO fact_payments
SELECT
    order_id,
    payment_sequential,
    payment_type,
    payment_installments,
    payment_value
FROM stg_payments;
""")

conn.commit()

#Criação de métricas e views

In [ ]:
cursor.executescript("""

-- metricas de orders

DROP VIEW IF EXISTS vw_orders_kpis;

CREATE VIEW vw_orders_kpis AS

WITH pagamentos AS (
    SELECT
      order_id,
      COALESCE(SUM(payment_value),0) AS total_payment_value
    FROM fact_payments
    GROUP BY order_id
),

valores_pedidos AS (
    SELECT
      o.order_id,
      o.order_status,
      COALESCE(SUM(oi.price + oi.freight_value),0) AS total_order_value,
      COALESCE(p.total_payment_value,0) AS total_payment_value,
      CASE
        WHEN o.order_approved_at IS NOT NULL AND o.order_purchase_timestamp IS NOT NULL THEN CAST(julianday(o.order_approved_at) - julianday(o.order_purchase_timestamp) AS INTEGER)
        ELSE NULL
      END AS approval_time,
      CASE
        WHEN o.order_approved_at IS NOT NULL AND o.order_delivered_carrier_date IS NOT NULL THEN CAST(julianday(o.order_delivered_carrier_date) - julianday(o.order_approved_at) AS INTEGER)
        ELSE NULL
      END AS shipping_lead_time,
      CASE
        WHEN o.order_delivered_customer_date IS NOT NULL AND o.order_delivered_carrier_date IS NOT NULL THEN CAST(julianday(o.order_delivered_customer_date) - julianday(o.order_delivered_carrier_date) AS INTEGER)
        ELSE NULL
      END AS delivery_lead_time,
      CASE
        WHEN o.order_delivered_customer_date IS NOT NULL AND o.order_purchase_timestamp IS NOT NULL THEN CAST(julianday(o.order_delivered_customer_date) - julianday(o.order_purchase_timestamp) AS INTEGER)
        ELSE NULL
      END AS order_lead_time,
      CASE
        WHEN oi.shipping_limit_date IS NOT NULL AND o.order_delivered_carrier_date IS NOT NULL THEN CAST(julianday(o.order_delivered_carrier_date) - julianday(oi.shipping_limit_date) AS INTEGER)
        ELSE NULL
      END AS shipping_delta,
      CASE
        WHEN o.order_delivered_customer_date IS NOT NULL AND o.order_estimated_delivery_date IS NOT NULL THEN CAST(julianday(o.order_delivered_customer_date) - julianday(o.order_estimated_delivery_date) AS INTEGER)
        ELSE NULL
      END AS delivery_delta
    FROM fact_orders o
    LEFT JOIN fact_order_items oi ON o.order_id = oi.order_id
    LEFT JOIN pagamentos p ON o.order_id = p.order_id
    GROUP BY o.order_id, o.order_status
)

SELECT
  order_id,
  order_status,
  total_order_value,
  total_payment_value,
  CASE
    WHEN order_status = 'canceled' AND total_payment_value > 0 THEN 'estorno necessario'
    WHEN order_status = 'canceled' AND total_payment_value = 0 THEN 'cancelado sem pagamento'
    WHEN order_status != 'canceled' AND total_payment_value = 0 THEN 'pendente'
    WHEN order_status != 'canceled' AND (total_payment_value - total_order_value) > 0.05 THEN 'pago indevidamente'
    WHEN order_status != 'canceled' AND (total_order_value - total_payment_value) > 0.05 THEN 'parcialmente pago'
    WHEN order_status != 'canceled' AND ABS(total_payment_value - total_order_value) <= 0.05 THEN 'totalmente pago'
    ELSE 'verificacao necessaria'
  END AS payment_status,
  approval_time,
  shipping_lead_time,
  delivery_lead_time,
  order_lead_time,
  shipping_delta,
  delivery_delta,
  CASE
    WHEN shipping_delta <= 0 THEN 1
    WHEN shipping_delta > 0 THEN 0
    ELSE NULL
  END AS shipped_on_time,
  CASE
    WHEN delivery_delta <= 0 THEN 1
    WHEN delivery_delta > 0 THEN 0
    ELSE NULL
  END AS delivered_on_time
  FROM valores_pedidos
  WHERE total_order_value > 0;

-- metricas de order_items

DROP VIEW IF EXISTS vw_orders_items_kpis;

CREATE VIEW vw_orders_items_kpis AS

SELECT
    oi.order_id,
    oi.order_item_id,
    oi.product_id,
    SUM(oi.price) AS total_price,
    SUM(oi.freight_value) AS total_freight,
    SUM(oi.price + oi.freight_value) AS total_amount
FROM fact_order_items oi
GROUP BY oi.order_id, oi.order_item_id, oi.product_id;

-- metricas de customers

DROP VIEW IF EXISTS vw_customers_kpis;

CREATE VIEW vw_customers_kpis AS

WITH customers_cte AS (
  SELECT
  c.customer_unique_id,
  COUNT(o.order_id) AS number_of_orders,
  COALESCE(SUM(oi.price + oi.freight_value),0) AS total_sold_client,
  MIN(o.order_purchase_timestamp) AS first_purchase
FROM dim_customers c
LEFT JOIN fact_orders o ON c.customer_unique_id = o.customer_unique_id
LEFT JOIN fact_order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_unique_id
)

SELECT
  customer_unique_id,
  number_of_orders,
  total_sold_client,
  CASE
    WHEN number_of_orders > 1 THEN 1
    ELSE 0
  END AS frequent_customer,
  CASE
    WHEN number_of_orders > 0 THEN ROUND(total_sold_client / number_of_orders, 2)
    ELSE NULL
  END AS avg_ticket_cust,
  first_purchase
FROM customers_cte;

-- metricas de sellers

DROP VIEW IF EXISTS vw_sellers_kpis;

CREATE VIEW vw_sellers_kpis AS

WITH seller_cte AS (
    SELECT
      s.seller_id,
      oi.order_id,
      oi.order_item_id,
      COALESCE(oi.price + oi.freight_value, 0) AS sold_value,
      CASE
        WHEN o.order_status = 'delivered' THEN
          CAST(julianday(o.order_delivered_customer_date) - julianday(o.order_purchase_timestamp) AS INTEGER)
        ELSE NULL
      END AS seller_delivery_time
    FROM dim_sellers s
    LEFT JOIN fact_order_items oi ON s.seller_id = oi.seller_id
    LEFT JOIN fact_orders o ON oi.order_id = o.order_id
)

SELECT
  seller_id,
  COUNT(DISTINCT order_id || '-' || order_item_id) AS total_items_sold,
  SUM(sold_value) AS total_sold_seller,
  AVG(seller_delivery_time) AS avg_delivery_time
FROM seller_cte
GROUP BY seller_id;

-- metricas de reviews

DROP VIEW IF EXISTS vw_reviews_kpis;

CREATE VIEW vw_reviews_kpis AS

SELECT
  r.review_id,
  CASE
    WHEN r.review_score IN (4,5) THEN 'promoter'
    WHEN r.review_score = 3 THEN 'passive'
    WHEN r.review_score IN (1,2) THEN 'detractor'
    ELSE NULL
  END AS nps_group

FROM fact_reviews r
""")

conn.commit()

#Exportação dos dados

In [ ]:
cursor.executescript("""
DROP TABLE IF EXISTS stg_customers;
DROP TABLE IF EXISTS stg_sellers;
DROP TABLE IF EXISTS stg_products;
DROP TABLE IF EXISTS stg_location;
DROP TABLE IF EXISTS stg_orders;
DROP TABLE IF EXISTS stg_order_items;
DROP TABLE IF EXISTS stg_reviews;
DROP TABLE IF EXISTS stg_payments;
""")

conn.commit()
conn.close()

files.download("olist.db")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>